In [4]:
import pickle
import sklearn
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [24]:
RANDOM_STATE = 42
RANDOM_MODEL = 17

In [1]:
!pip install lightgbm

    100% |################################| 624kB 2.3MB/s eta 0:00:01


In [2]:
import lightgbm as lgb

In [42]:
!pip install catboost

    100% |################################| 19.5MB 74kB/s  eta 0:00:01               | 6.7MB 69.6MB/s eta 0:00:01


In [44]:
import catboost

In [5]:
X_train_full = pickle.load(open('x_train.pkl', 'rb'))

In [6]:
X_submit = pickle.load(open('x_test.pkl', 'rb'))
y_train_full = pickle.load(open('y_train.pkl', 'rb'))

In [7]:
y_train_labels = y_train_full.driver_response.values

In [56]:
X_train_full_prep = X_train_full.drop(["offer_gk"], axis=1)
X_submit_prep = X_submit.drop(["offer_gk"], axis=1)

In [18]:
x_train, x_test, y_train, y_test = train_test_split(X_train_full, y_train_labels, test_size=0.26, random_state=RANDOM_STATE)

In [19]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(660492, 149) (660492,) (232065, 149) (232065,)


In [25]:
lr = LogisticRegression(random_state=RANDOM_MODEL)
lr.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [26]:
y_predict=lr.predict_proba(x_test)

In [27]:
y_predict[:,1]

array([ 0.62924216,  0.56862183,  0.50971579, ...,  0.81599951,
        0.54589683,  0.73668557])

In [28]:
roc_auc_score(y_test, y_predict[:,1])

0.50070353951439239

In [32]:
lgbc = lgb.LGBMClassifier(random_state=RANDOM_STATE)
lgbc.fit(x_train, y_train)

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/lightgbm/basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/lightgbm/basic.py:648: LGBMDeprecationWarning: The `max_bin` parameter is deprecated and will be removed in 2.0.12 version. Please use `params` to pass this parameter.
  'Please use `params` to pass this parameter.', LGBMDeprecationWarning)


LGBMClassifier(boosting_type='gbdt', colsample_bytree=1.0, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=10,
        n_jobs=-1, num_leaves=31, objective=None, random_state=42,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1)

In [37]:
lgb_pred=lgbc.predict_proba(x_test)[:,1]
lgb_pred_train=lgbc.predict_proba(x_train)[:,1]

In [41]:
print("train ROCAUC", roc_auc_score(y_train, lgb_pred_train))
print("test ROCAUC", roc_auc_score(y_test, lgb_pred))

train ROCAUC 0.740796378173
test ROCAUC 0.74025480981


In [57]:
lgbc.fit(X_train_full_prep, y_train_labels)

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/lightgbm/basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/lightgbm/basic.py:648: LGBMDeprecationWarning: The `max_bin` parameter is deprecated and will be removed in 2.0.12 version. Please use `params` to pass this parameter.
  'Please use `params` to pass this parameter.', LGBMDeprecationWarning)


LGBMClassifier(boosting_type='gbdt', colsample_bytree=1.0, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=10,
        n_jobs=-1, num_leaves=31, objective=None, random_state=42,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1)

In [58]:
roc_auc_score(y_train_labels, lgbc.predict_proba(X_train_full_prep)[:,1])

0.74083901360040028

In [59]:
final_predict=lgbc.predict_proba(X_submit_prep)[:,1]

In [68]:
submission = pd.DataFrame()
submission['offer_gk']=X_submit['offer_gk'].values
submission['driver_response']=final_predict
submission.set_index('offer_gk', inplace=True)

In [69]:
submission.to_csv('submission.csv')

In [70]:
!head -n 4 submission.csv

offer_gk,driver_response
152446,0.7197967127684622
281031,0.6424696878693619
779964,0.7280778149357617
